In [1]:
import os
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from torchvision import models,transforms
import torch.optim as optim
from sklearn.model_selection import train_test_split

In [2]:
# Function to read temperature values from a file
def read_temperature_file(filepath):
    try:
        with open(filepath, 'r') as file:
            lines = file.readlines()
            data = []
            for line in lines:
                # Clean and split the line
                line_data = line.strip().split()
                # Convert to float and append
                data.extend([float(val) for val in line_data])
            return np.array(data)
    except Exception as e:
        print(f"Error reading {filepath}: {e}")
        return None

In [3]:
# Function to load temperature data and labels from a directory
def load_data_from_directory(directory, label):
    data = []
    labels = []
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        if filepath.endswith('.txt'):
            temp_values = read_temperature_file(filepath)
            if temp_values is not None:
                data.append(temp_values)
                labels.append(label)
    return data, labels

In [4]:
training_healthy = '/kaggle/input/breast-cancer-temp-values/Breast_cancer_temp_values 2/Breast_cancer_temp_values/Single/Healthy/Training'
testing_healthy = '/kaggle/input/breast-cancer-temp-values/Breast_cancer_temp_values 2/Breast_cancer_temp_values/Single/Healthy/Testing'
training_sick = '/kaggle/input/breast-cancer-temp-values/Breast_cancer_temp_values 2/Breast_cancer_temp_values/Single/Sick/training'
testing_sick = '/kaggle/input/breast-cancer-temp-values/Breast_cancer_temp_values 2/Breast_cancer_temp_values/Single/Sick/Testing'

In [5]:
# Load data and labels
train_data_healthy, train_labels_healthy = load_data_from_directory(training_healthy, 0)
train_data_sick, train_labels_sick = load_data_from_directory(training_sick, 1)
test_data_healthy, test_labels_healthy = load_data_from_directory(testing_healthy, 0)
test_data_sick, test_labels_sick = load_data_from_directory(testing_sick, 1)

# Combine healthy and sick data
train_data = train_data_healthy + train_data_sick
train_labels = train_labels_healthy + train_labels_sick
test_data = test_data_healthy + test_data_sick
test_labels = test_labels_healthy + test_labels_sick

# Convert lists to numpy arrays
train_data = np.array(train_data)
train_labels = np.array(train_labels)
test_data = np.array(test_data)
test_labels = np.array(test_labels)

In [6]:
# Split the training data into training and validation sets (80-20 split)
train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)

print(f"Train data shape: {train_data.shape}")
print(f"Validation data shape: {val_data.shape}")
print(f"Test data shape: {test_data.shape}")

Train data shape: (37, 307200)
Validation data shape: (10, 307200)
Test data shape: (10, 307200)


In [7]:
# Define a PyTorch dataset
class TemperatureDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = torch.tensor(data, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.float32)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        sample = sample.view(1, -1, 1)  # Reshape to (1, number of temperature values, 1)
        if self.transform:
            sample = self.transform(sample)
        label = self.labels[idx]
        return sample, label

In [8]:
# Define the transform to resize and convert to 3 channels
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1)),  # Repeat the single channel to get 3 channels
])

# Create datasets and dataloaders
train_dataset = TemperatureDataset(train_data, train_labels, transform=transform)
val_dataset = TemperatureDataset(val_data, val_labels, transform=transform)
test_dataset = TemperatureDataset(test_data, test_labels, transform=transform)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [9]:
# Load the pre-trained DenseNet121 model
model = models.densenet121(pretrained=True)

# Modify the classifier to fit our task
num_features = model.classifier.in_features
model.classifier = nn.Sequential(
    nn.Linear(num_features, 1),
    nn.Sigmoid()
)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 82.2MB/s]


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [10]:
# Train and evaluate the model
num_epochs = 20
best_val_accuracy = 0.0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for data, labels in train_dataloader:
        data, labels = data.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(data).squeeze()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    train_loss = running_loss / len(train_dataloader)

    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for data, labels in val_dataloader:
            data, labels = data.to(device), labels.to(device)
            outputs = model(data).squeeze()
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            predicted = (outputs > 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_loss = val_loss / len(val_dataloader)
    val_accuracy = 100 * correct / total

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")

     # Save the best model based on validation accuracy
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save(model.state_dict(), 'best_model_densenet121.pth')

/opt/conda/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Epoch 1/20, Train Loss: 0.5549, Val Loss: 0.5302, Val Accuracy: 80.00%
Epoch 2/20, Train Loss: 0.4061, Val Loss: 0.4632, Val Accuracy: 80.00%
Epoch 3/20, Train Loss: 0.2186, Val Loss: 0.4417, Val Accuracy: 80.00%
Epoch 4/20, Train Loss: 0.1190, Val Loss: 0.4709, Val Accuracy: 80.00%
Epoch 5/20, Train Loss: 0.2271, Val Loss: 0.4881, Val Accuracy: 80.00%
Epoch 6/20, Train Loss: 0.1959, Val Loss: 0.4198, Val Accuracy: 80.00%
Epoch 7/20, Train Loss: 0.2155, Val Loss: 0.6283, Val Accuracy: 70.00%
Epoch 8/20, Train Loss: 0.1201, Val Loss: 0.9481, Val Accuracy: 80.00%
Epoch 9/20, Train Loss: 0.1228, Val Loss: 1.3451, Val Accuracy: 80.00%
Epoch 10/20, Train Loss: 0.0343, Val Loss: 1.6075, Val Accuracy: 80.00%
Epoch 11/20, Train Loss: 0.1557, Val Loss: 1.9224, Val Accuracy: 80.00%
Epoch 12/20, Train Loss: 0.1055, Val Loss: 1.5145, Val Accuracy: 80.00%
Epoch 13/20, Train Loss: 0.0308, Val Loss: 1.5634, Val Accuracy: 90.00%
Epoch 14/20, Train Loss: 0.0461, Val Loss: 1.1219, Val Accuracy: 90.00%
E

In [11]:
# Load the best model
model.load_state_dict(torch.load('best_model_densenet121.pth'))

# Evaluate on the test set
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data, labels in test_dataloader:
        data, labels = data.to(device), labels.to(device)
        outputs = model(data).squeeze()
        predicted = (outputs > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
test_accuracy = 100 * correct / total
print(f"Accuracy on test set: {test_accuracy:.2f}%")

Accuracy on test set: 60.00%
